# Load song and display spectrogram

In [ ]:
%matplotlib inline
import librosa
import matplotlib.pyplot as plt
import matplotlib
import librosa.display as display
import numpy as np
import IPython.display as ipd
from collections import OrderedDict
import scipy

In [ ]:
y, sr = librosa.load('../../data/latch.mp3', duration=30)
D = librosa.stft(y, n_fft=2048)

In [ ]:
matplotlib.rcParams['figure.figsize'] = [20, 5]

od = OrderedDict()
od['D'] = D

# Plot
index = 1
plt.figure()
for key, value in od.items(): 
    plt.subplot(len(od), 1, index)
    display.specshow(librosa.amplitude_to_db(np.abs(value), ref=np.max), y_axis='log')
    #display.specshow(np.abs(value), y_axis='log')
    plt.title('Full power spectrogram: {}'.format(key))
    plt.colorbar(format='%+2.0f dB')
    index += 1 

plt.show();

# Exploring n_fft - related to number of rows in the STFT

In [ ]:
D8 = librosa.stft(y, n_fft=8)
D64 = librosa.stft(y, n_fft=64)
D128 = librosa.stft(y, n_fft=128)
D256 = librosa.stft(y, n_fft=256)
D512 = librosa.stft(y, n_fft=256)

In [ ]:
matplotlib.rcParams['figure.figsize'] = [15, 15]

od = OrderedDict()
od['D8'] = D8
od['D64'] = D64
od['D128'] = D128
od['D256'] = D256
od['D512'] = D512
od['D'] = D

# Plot
index = 1
plt.figure()
for key, value in od.items(): 
    plt.subplot(len(od), 1, index)
    display.specshow(librosa.amplitude_to_db(np.abs(value), ref=np.max), y_axis='log')
    plt.title('Full power spectrogram: {}'.format(key))
    plt.colorbar(format='%+2.0f dB')
    index += 1 

plt.show();

# Harmonic/Percussion separation

In [ ]:
DFull = librosa.stft(y, n_fft=2048)
DH, DP = librosa.decompose.hpss(D, margin=2)

matplotlib.rcParams['figure.figsize'] = [15, 15]

od = OrderedDict()
od['D'] = D
od['DHarmonic'] = DH
od['DPercussion'] = DP

# Plot
index = 1
plt.figure()
for key, value in od.items(): 
    plt.subplot(len(od), 1, index)
    display.specshow(librosa.amplitude_to_db(np.abs(value), ref=np.max), y_axis='log')
    plt.title('Full power spectrogram: {}'.format(key))
    plt.colorbar(format='%+2.0f dB')
    index += 1 

plt.show();

In [ ]:
librosa.amplitude_to_db(np.abs(D), ref=np.max).shape

In [ ]:
y_hat = librosa.istft(D)
yh_hat = librosa.istft(DH)
yp_hat = librosa.istft(DP)

In [ ]:
ipd.Audio(y, rate=sr)

In [ ]:
ipd.Audio(yh_hat, rate=sr)

In [ ]:
ipd.Audio(yp_hat, rate=sr)

# Melspectrogram

In [ ]:
# Generate melspectrogram
MS = librosa.feature.melspectrogram(S=DFull, sr=sr)
plt.figure(figsize=(10, 4))
S_dB = librosa.power_to_db(MS, ref=np.max)

# Plot some stuffs
plt.figure()
plt.subplot(2, 1, 1)
librosa.display.specshow(S_dB, x_axis='time',
                         y_axis='mel', sr=sr,
                         fmax=8000)
plt.subplot(2, 1, 2)
display.specshow(librosa.amplitude_to_db(np.abs(DFull), ref=np.max), y_axis='log')

# Explore chromagrams

In [ ]:
S = np.abs(librosa.stft(y, n_fft=4096))**2
chroma = librosa.feature.chroma_stft(S=S, sr=sr)
plt.figure(figsize=(10, 4))
librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('Chromagram')
plt.tight_layout()

In [ ]:
#np.abs(value)
librosa.amplitude_to_db(np.abs(value), ref=np.max)

In [ ]:
CQT = librosa.amplitude_to_db(np.abs(librosa.cqt(y, sr=sr)), ref=np.max)
librosa.display.specshow(CQT, y_axis='cqt_note')
plt.colorbar(format='%+2.0f dB')
plt.title('Constant-Q power spectrogram (note)')

# Foreground/ background separation

In [ ]:

# And compute the spectrogram magnitude and phase
S_full, phase = librosa.magphase(librosa.stft(y))


#######################################
# Plot a 5-second slice of the spectrum
plt.figure(figsize=(12, 4))
librosa.display.specshow(librosa.amplitude_to_db(S_full, ref=np.max),
                         y_axis='log', x_axis='time', sr=sr)
plt.colorbar()
plt.tight_layout()

###########################################################
# The wiggly lines above are due to the vocal component.
# Our goal is to separate them from the accompanying
# instrumentation.
#

# We'll compare frames using cosine similarity, and aggregate similar frames
# by taking their (per-frequency) median value.
#
# To avoid being biased by local continuity, we constrain similar frames to be
# separated by at least 2 seconds.
#
# This suppresses sparse/non-repetetitive deviations from the average spectrum,
# and works well to discard vocal elements.

S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sr)))

# The output of the filter shouldn't be greater than the input
# if we assume signals are additive.  Taking the pointwise minimium
# with the input spectrum forces this.
S_filter = np.minimum(S_full, S_filter)


##############################################
# The raw filter output can be used as a mask,
# but it sounds better if we use soft-masking.

# We can also use a margin to reduce bleed between the vocals and instrumentation masks.
# Note: the margins need not be equal for foreground and background separation
margin_i, margin_v = 2, 10
power = 2

mask_i = librosa.util.softmask(S_filter,
                               margin_i * (S_full - S_filter),
                               power=power)

mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)

# Once we have the masks, simply multiply them with the input spectrum
# to separate the components

S_foreground = mask_v * S_full
S_background = mask_i * S_full


##########################################
# Plot the same slice, but separated into its foreground and background

# sphinx_gallery_thumbnail_number = 2

plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
librosa.display.specshow(librosa.amplitude_to_db(S_full, ref=np.max),
                         y_axis='log', sr=sr)
plt.title('Full spectrum')
plt.colorbar()

plt.subplot(3, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(S_background, ref=np.max),
                         y_axis='log', sr=sr)
plt.title('Background')
plt.colorbar()
plt.subplot(3, 1, 3)
librosa.display.specshow(librosa.amplitude_to_db(S_foreground, ref=np.max),
                         y_axis='log', x_axis='time', sr=sr)
plt.title('Foreground')
plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
y_fg = librosa.istft(S_foreground*phase)
ipd.Audio(y_fg, rate=sr)

In [ ]:
y_bg = librosa.istft(S_background*phase)
ipd.Audio(y_bg, rate=sr)

In [ ]:
y_full = librosa.istft(S_full*phase)
ipd.Audio(y_full, rate=sr)

In [ ]:
ipd.Audio(y, rate=sr)

# Enhanced chroma

In [ ]:
#######################################
# First, let's plot the original chroma
chroma_orig = librosa.feature.chroma_cqt(y=y, sr=sr)

# And for comparison, we'll show the CQT matrix as well.
C = np.abs(librosa.cqt(y=y, sr=sr, bins_per_octave=12*3, n_bins=7*12*3))


plt.figure(figsize=(12, 4))
plt.subplot(2, 1, 1)
librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                         y_axis='cqt_note', bins_per_octave=12*3)
plt.colorbar()
plt.subplot(2, 1, 2)
librosa.display.specshow(chroma_orig, y_axis='chroma')
plt.colorbar()
plt.ylabel('Original')
plt.tight_layout()


###########################################################
# We can correct for minor tuning deviations by using 3 CQT
# bins per semi-tone, instead of one
chroma_os = librosa.feature.chroma_cqt(y=y, sr=sr, bins_per_octave=12*3)


plt.figure(figsize=(12, 4))

plt.subplot(2, 1, 1)
librosa.display.specshow(chroma_orig, y_axis='chroma')
plt.colorbar()
plt.ylabel('Original')


plt.subplot(2, 1, 2)
librosa.display.specshow(chroma_os, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.ylabel('3x-over')
plt.tight_layout()


########################################################
# That cleaned up some rough edges, but we can do better
# by isolating the harmonic component.
# We'll use a large margin for separating harmonics from percussives
y_harm = librosa.effects.harmonic(y=y, margin=8)
chroma_os_harm = librosa.feature.chroma_cqt(y=y_harm, sr=sr, bins_per_octave=12*3)


plt.figure(figsize=(12, 4))

plt.subplot(2, 1, 1)
librosa.display.specshow(chroma_os, y_axis='chroma')
plt.colorbar()
plt.ylabel('3x-over')

plt.subplot(2, 1, 2)
librosa.display.specshow(chroma_os_harm, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.ylabel('Harmonic')
plt.tight_layout()


###########################################
# There's still some noise in there though.
# We can clean it up using non-local filtering.
# This effectively removes any sparse additive noise from the features.
chroma_filter = np.minimum(chroma_os_harm,
                           librosa.decompose.nn_filter(chroma_os_harm,
                                                       aggregate=np.median,
                                                       metric='cosine'))


plt.figure(figsize=(12, 4))

plt.subplot(2, 1, 1)
librosa.display.specshow(chroma_os_harm, y_axis='chroma')
plt.colorbar()
plt.ylabel('Harmonic')

plt.subplot(2, 1, 2)
librosa.display.specshow(chroma_filter, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.ylabel('Non-local')
plt.tight_layout()


###########################################################
# Local discontinuities and transients can be suppressed by
# using a horizontal median filter.
chroma_smooth = scipy.ndimage.median_filter(chroma_filter, size=(1, 9))


plt.figure(figsize=(12, 4))

plt.subplot(2, 1, 1)
librosa.display.specshow(chroma_filter, y_axis='chroma')
plt.colorbar()
plt.ylabel('Non-local')

plt.subplot(2, 1, 2)
librosa.display.specshow(chroma_smooth, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.ylabel('Median-filtered')
plt.tight_layout()


#########################################################
# A final comparison between the CQT, original chromagram
# and the result of our filtering.
plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                         y_axis='cqt_note', bins_per_octave=12*3)
plt.colorbar()
plt.ylabel('CQT')
plt.subplot(3, 1, 2)
librosa.display.specshow(chroma_orig, y_axis='chroma')
plt.ylabel('Original')
plt.colorbar()
plt.subplot(3, 1, 3)
librosa.display.specshow(chroma_smooth, y_axis='chroma', x_axis='time')
plt.ylabel('Processed')
plt.colorbar()
plt.tight_layout()
plt.show()